## Setup

In [1]:
# Basic Setup

import pandas as pd
import numpy as np 
import sklearn 
import os

"""
This data is from the 2020-2021 and 2021-2022 seasons.
The data was scraped partway through the 2021-2022 season, so we won't have the complete match history for the season.

"""

matches = pd.read_csv('../data/raw/matches.csv')

matches.index = matches[matches.columns[0]]
matches.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


## Exploration

In [2]:
# Inspect matches 

# matches.info()
# matches.team.value_counts()
# matches.comp.value_counts()
# matches['match report'].value_counts()
# matches.isna().sum()/len(matches.index)*100
matches.dtypes
matches.result.value_counts()

L    548
W    526
D    315
Name: result, dtype: int64

#### Initial Notes

* The notes columns is completely blank. To be dropped.
* The attendance column is half empty. Not reliable to be used as a predictor. 
* Upon inspecting the columns, it is noted: 

    1. The columns comp, & match report only hold one value i.e 'premier league' and 'match report'.  These values hold no   
    bearing on the outcome of a football match and hence are to be removed. 

* Convert date & time columns to the appropriate datatype. 
* Handle categorical columns by converting them to a numerical type

 P.S: This is just an initial screening. We might drop more columns as we decide what columns to be used as predictors. 


    We have data for 2 seasons. There are 20 teams and each team plays 38 matches in a season. 
    Therefore, we should have data for 2*20*38 = 1520 matches.

    At the end of each seaoson, 3 teams are relegated and 3 teams are promoted, so we'll have:

    - 6 teams with 38 matches each
    - 17 teams with 76 matches each

    Let's now inspect the following things:

    1. How many matches we have the data for?
    2. How many rows are missing due to the data being scraped partway through the season?
    3. Are there any teams that are missing more data than we'd expect?

In [3]:
# Since each row in the dataframe corresponds to on match, To determine how many matches we have data for, we just need to find the length
# of the dataframe

num_matches = len(matches.index)
num_matches

# Number of missing rows. 

matches['date'] = pd.to_datetime(matches.date)
# matches.date.min()


In [4]:
# Exploring team wise match data 

team_matches = matches.team.value_counts()
team_matches


Southampton                 72
Brighton and Hove Albion    72
Manchester United           72
West Ham United             72
Newcastle United            72
Burnley                     71
Leeds United                71
Crystal Palace              71
Manchester City             71
Wolverhampton Wanderers     71
Tottenham Hotspur           71
Arsenal                     71
Leicester City              70
Chelsea                     70
Aston Villa                 70
Everton                     70
Liverpool                   38
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Brentford                   34
Watford                     33
Norwich City                33
Name: team, dtype: int64

#### Observations

* We have data for 1389 matches.
* We have data missing for 131 matches.
* Liverpool has data for 38 matches in our dataset. Since, it wasn't relegated or promoted in the the 2020-2021 season.  
 There's some match data missing from Liverpool.

#### Next Steps 

Since only about 8% of the data is missing, we will proceed with the analysis with the data that is available. 


# Cleaning & Transformation 

Acting on the initial notes, we shall be doing the following:

1. Dropping columns: 1. notes 2. comp. 3. match report 4. attendance. 
2. Run one hot encoding on the categorical columns 

In [5]:
#  Dropping uninformative columns 

matches.drop(['comp', 'match report', 'attendance', 'notes', 'Unnamed: 0','round'], axis = 1, inplace = True)



In [6]:
len(matches.index)

1389

In [7]:
# Handling datetime 

import datetime
from datetime import date, time 

matches['date'] = pd.to_datetime(matches.date).dt.date
matches['time'] = pd.to_datetime(matches.time).dt.time
matches['datetime'] = matches.apply(lambda r : pd.datetime.combine(r['date'],r['time']),1)


/var/folders/ql/4dbklqz57h5c3nsx984r5c840000gn/T/ipykernel_3488/1395765472.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  matches['datetime'] = matches.apply(lambda r : pd.datetime.combine(r['date'],r['time']),1)


In [17]:
# Handling Categoricals 

matches['venue']  =   pd.Categorical(matches.venue, ordered = False)
matches['opponent'] = pd.Categorical(matches.opponent, ordered = False)
matches['day'] =      pd.Categorical(matches.day, ordered = False)
matches['hour'] =     matches.datetime.apply(lambda x: x.hour)
matches['formation'] = pd.Categorical(matches.formation, ordered = False)
matches['result'] =  pd.Categorical(matches.result, ordered = False)

matches.venue = matches.venue.cat.codes
matches.opponent = matches.opponent.cat.codes
matches.day = matches.day.cat.codes
matches.formation = matches.formation.cat.codes
matches.result = matches.result.cat.codes



# matches['venue'] = matches.venue.cat.codes
# matches['opponent'] 
# Droppping uninformative columns 

matches.drop(['referee', 'captain'], axis = 1, inplace = True)

# Separating all categorical columns 

matches.hour = matches.hour.astype('int8')
matches_cats = matches.select_dtypes(include = 'int8').drop('result', axis = 1)
matches_cats[['datetime', 'team']] = matches[['datetime', 'team']]

# Creating a copy of matches at this stage 

matches_cat_transforms = matches

In [18]:
matches_backup = matches

group = matches.groupby('team')
group = group.get_group('Manchester City')
# group = group.sort_values('date')
# rolling_stats = group(cols).rolling(3, closed = 'left').mean()
# group[new_cols] = rolling_stats
# group = group.dropna(subset = new_cols)


In [19]:
# Adding Rolling Statistics 

def get_rolling_avg(col, new_cols, group):
    
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    
    return group

    
cols = ["gf","ga","sh","sot","dist","fk","pk","pkatt"]
new_cols = [f"{c}_rolling" for c in cols ]
    

In [20]:
matches_rolling  = matches.groupby('team').apply(lambda x: get_rolling_avg(cols, new_cols,x))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.index.shape[0])

# Separating out the rolling columns

matches_rolling_stats = matches_rolling[new_cols]
matches_rolling_stats[['datetime','team']] = matches_rolling[['datetime', 'team']]




/var/folders/ql/4dbklqz57h5c3nsx984r5c840000gn/T/ipykernel_59451/3596522911.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_rolling_stats[['datetime','team']] = matches_rolling[['datetime', 'team']]
/var/folders/ql/4dbklqz57h5c3nsx984r5c840000gn/T/ipykernel_59451/3596522911.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_rolling_stats[['datetime','team']] = matches_rolling[['datetime', 'team']]


In [21]:
matches_predictors = pd.merge(matches_cats,matches_rolling_stats, how = 'inner', on = ['datetime','team'])
matches_predictors.index = matches_predictors[['datetime','team']]

In [22]:
matches_target = matches[['datetime','team','result']]
matches_target['target'] = [1 if i  == 2 else 0 for i in matches_target.result.tolist()]
matches_target.drop('result', axis = 1, inplace = True)
matches_target.index = matches_target[['datetime','team']]

/var/folders/ql/4dbklqz57h5c3nsx984r5c840000gn/T/ipykernel_59451/3676419324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_target['target'] = [1 if i  == 2 else 0 for i in matches_target.result.tolist()]
/var/folders/ql/4dbklqz57h5c3nsx984r5c840000gn/T/ipykernel_59451/3676419324.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_target.drop('result', axis = 1, inplace = True)


In [23]:
matches_utils = matches[['season', 'datetime', 'team']]
matches_utils.index = matches_utils[['datetime','team']]

In [24]:
matches_interim = pd.merge(matches_predictors, matches_target, how = 'inner', on = ['datetime','team']).merge(matches_utils, how = 'inner', 
                            on = ['datetime','team']) 

# Storing the prepared dataset. 

import pickle

matches_interim.to_pickle('../data/interim/matches_interim.pkl')
matches_interim.to_csv('../data/interim/matches_interim.csv')